<a href="https://colab.research.google.com/github/vipashaaV321/User-Intent-Modeling/blob/main/failed_approaches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.14.0
Uninstalling tensorflow-2.14.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.14.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? Y
Y
  Successfully uninstalled tensorflow-2.14.0


In [ ]:
!pip install tensorflow-gpu

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from zipfile import ZipFile
from urllib.request import urlretrieve
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score, recall_score
from sklearn.decomposition import NMF

In [ ]:
## extracting data
urlretrieve("http://files.grouplens.org/datasets/movielens/ml-1m.zip", "movielens.zip")
ZipFile("movielens.zip", "r").extractall()

In [ ]:
ratings = pd.read_csv(
    "ml-1m/ratings.dat",
    sep="::",
    names=["user_id", "item_id", "rating", "unix_timestamp"],
)
ratings.head()

<ipython-input-39-b72da1ac23ec>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv(


,user_id,item_id,rating,unix_timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
ratings = ratings.drop("unix_timestamp", axis=1)

In [ ]:
ratings.head()

,user_id,item_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [ ]:

# Prepare user profiles and item importance weights
user_profiles = {}
item_importance_weights = tf.Variable(tf.ones(len(ratings['item_id'].unique())))

for user_id, item_id, rating in ratings.values:
    if user_id not in user_profiles:
        user_profiles[user_id] = {}
    user_profiles[user_id][item_id] = rating

In [ ]:
 tf.experimental.numpy.experimental_enable_numpy_behavior()

In [ ]:
# Define attention modeling with feedback-based neural network
def attention_model(user_profile, item_ids, item_importance_weights):

    tf.compat.v1.disable_eager_execution()
    # Convert user profile to a tensor
    print("Inside Attention Modeling")
    user_profile_values = list(user_profile.values())
    print("User Profile Values",user_profile_values)
    user_profile_tensor = tf.convert_to_tensor(user_profile_values)
    user_profile_tensor= tf.cast(user_profile_tensor, tf.float32)
    user_profile_tensor = tf.expand_dims(user_profile_tensor, axis=0)
    print("User Profile Tensor",user_profile_tensor)
    # user_profile_tensor = tf.eval(user_profile_tensor)
    print(user_profile_tensor.shape)
    # Embed item IDs
    item_embedding_layer = tf.keras.layers.Embedding(input_dim=len(ratings['item_id'].unique()), output_dim=16)
    item_embeddings = item_embedding_layer(item_ids)
    item_embeddings = tf.transpose(item_embeddings)

    print("Item EMb",item_embeddings)
    print(item_embeddings.shape)

    # Calculate attention scores using neural network
    attention_scores = tf.matmul(user_profile_tensor, item_embeddings.T)
    print("attn:",attention_scores)
    attention_scores = tf.nn.softmax(attention_scores)
    print("soft att",attention_scores)

    # Define item importance weights with explicit data type
    item_importance_weights = tf.Variable(tf.ones(len(ratings['item_id'].unique()), dtype=tf.float32))

    # Convert item importance weights to float32 data type
    item_importance_weights_float32 = tf.cast(item_importance_weights, tf.float32)

    # Calculate attention scores using neural network
    attention_scores = tf.nn.softmax(item_importance_weights_float32)

  # Apply item importance weights
    weighted_attention_scores = attention_scores * item_importance_weights_float32


    # Convert item importance weights to float32 data type
    # item_importance_weights_float32 = tf.cast(item_importance_weights, tf.float32)

    # # Calculate weighted attention scores
    # weighted_attention_scores = attention_scores * item_importance_weights_float32
    # Apply item importance weights
    # weighted_attention_scores = attention_scores * item_importance_weights
    print("w att:",weighted_attention_scores)

    return weighted_attention_scores

In [ ]:


# Define multi-round recommendation function
def multi_round_recommendation(user_id, num_recommendations, num_rounds):
    print("inside multround")
    recommendations = []

    is_finished = False
    for _ in range(num_rounds):
        # Get user profile for current round
        current_user_profile = user_profiles[user_id]
        print(user_profiles[user_id])
        print(current_user_profile)

        # Recommend items based on attention model
        item_ids = np.array(list(current_user_profile.keys()))
        item_ratings = np.array(list(current_user_profile.values()))

        print("itemID",item_ids)
        print("itemRatings",item_ratings)


        # Update item importance weights using neural network
        with tf.GradientTape() as tape:
            print("in a loop of item importance updation")
            attention_scores = attention_model(current_user_profile, item_ids, item_importance_weights)
            print("Attention Scores from a func")
            print(attention_scores)
            print(attention_scores.dtype)
            print(item_importance_weights)
            item_importance_gradients = tape.gradient(attention_scores, item_importance_weights)
            print("Grad")
            print(item_importance_gradients)

        # Update item importance weights based on feedback
        for item_id, item_rating, item_importance_gradient in zip(item_ids, item_ratings, item_importance_gradients):
            item_importance_weights.assign_sub(item_importance_gradient * item_rating)

        # Recommend top n items
        recommended_item_ids = item_ids[np.argsort(attention_scores)[-num_recommendations:]]

        print("Recommend: ",recommended_item_ids)

        # Print recommended items
        print("Recommended items:")
        for item_id in recommended_item_ids:
            print(item_id)

        # Get user feedback
        selected_item_id = input("Select an item (or type 'end' to finish): ")

        if selected_item_id == 'end':
            is_finished = True
            break

        # Update user profile with new feedback
        current_user_profile[selected_item_id] = 1

        # Add recommended items to the list
        recommendations.extend(recommended_item_ids)

    if not is_finished:
        print("Recommendations:")
        for item_id in recommendations:
            print(item_id)


In [ ]:

# Example usage
user_id = 1
num_recommendations = 10
num_rounds = 3

multi_round_recommendation(user_id, num_recommendations, num_rounds)

inside multround
{1193: 5, 661: 3, 914: 3, 3408: 4, 2355: 5, 1197: 3, 1287: 5, 2804: 5, 594: 4, 919: 4, 595: 5, 938: 4, 2398: 4, 2918: 4, 1035: 5, 2791: 4, 2687: 3, 2018: 4, 3105: 5, 2797: 4, 2321: 3, 720: 3, 1270: 5, 527: 5, 2340: 3, 48: 5, 1097: 4, 1721: 4, 1545: 4, 745: 3, 2294: 4, 3186: 4, 1566: 4, 588: 4, 1907: 4, 783: 4, 1836: 5, 1022: 5, 2762: 4, 150: 5, 1: 5, 1961: 5, 1962: 4, 2692: 4, 260: 4, 1028: 5, 1029: 5, 1207: 4, 2028: 5, 531: 4, 3114: 4, 608: 4, 1246: 4}
{1193: 5, 661: 3, 914: 3, 3408: 4, 2355: 5, 1197: 3, 1287: 5, 2804: 5, 594: 4, 919: 4, 595: 5, 938: 4, 2398: 4, 2918: 4, 1035: 5, 2791: 4, 2687: 3, 2018: 4, 3105: 5, 2797: 4, 2321: 3, 720: 3, 1270: 5, 527: 5, 2340: 3, 48: 5, 1097: 4, 1721: 4, 1545: 4, 745: 3, 2294: 4, 3186: 4, 1566: 4, 588: 4, 1907: 4, 783: 4, 1836: 5, 1022: 5, 2762: 4, 150: 5, 1: 5, 1961: 5, 1962: 4, 2692: 4, 260: 4, 1028: 5, 1029: 5, 1207: 4, 2028: 5, 531: 4, 3114: 4, 608: 4, 1246: 4}
itemID [1193  661  914 3408 2355 1197 1287 2804  594  919  595  938

TypeError: ignored

In [ ]:
# Prepare user profiles and item importance weights
user_profiles = {}
item_importance_weights = tf.Variable(tf.ones(len(ratings['item_id'].unique())))

for user_id, item_id, rating in ratings.values:
    if user_id not in user_profiles:
        user_profiles[user_id] = {}
    user_profiles[user_id][item_id] = rating

# Define attention modeling with feedback-based neural network
def attention_model(user_profile_tensor, item_ids, item_importance_weights):
    # Embed item IDs
    item_embedding_layer = tf.keras.layers.Embedding(input_dim=len(ratings['item_id'].unique()), output_dim=16)
    item_embeddings = item_embedding_layer(item_ids)

    # Calculate attention scores using neural network
    attention_scores = tf.matmul(user_profile_tensor, item_embeddings.T)
    attention_scores = tf.nn.softmax(attention_scores)

    # Apply item importance weights
    weighted_attention_scores = attention_scores * item_importance_weights

    return weighted_attention_scores

# Define multi-round recommendation function
def multi_round_recommendation(user_id, num_recommendations, num_rounds):
    recommendations = []

    is_finished = False
    for _ in range(num_rounds):
        # Get user profile for current round
        current_user_profile = user_profiles[user_id]

        # Convert user profile to a tensor
        user_profile_values = list(current_user_profile.values())
        user_profile_tensor = tf.convert_to_tensor(user_profile_values)

        # Recommend items based on attention model
        item_ids = np.array(list(current_user_profile.keys()))
        item_ratings = np.array(list(current_user_profile.values()))

        # Update item importance weights using neural network
        with tf.GradientTape() as tape:
            attention_scores = attention_model(user_profile_tensor, item_ids, item_importance_weights)
            item_importance_gradients = tape.gradient(attention_scores, item_importance_weights)

        # Update item importance weights based on feedback
        for item_id, item_rating, item_importance_gradient in zip(item_ids, item_ratings, item_importance_gradients):
            item_importance_weights.assign_sub(item_importance_gradient * item_rating)

        # Recommend top n items
        recommended_item_ids = item_ids[np.argsort(attention_scores)[-num_recommendations:]]

        # Print recommended items
        print("Recommended items:")
        for item_id in recommended_item_ids:
            print(item_id)

        # Get user feedback
        selected_item_id = input("Select an item (or type 'end' to finish): ")

        if selected_item_id == 'end':
            is_finished = True
            break

        # Update user profile with new feedback
        current_user_profile[selected_item_id] = 1

        # Add recommended items to the list
        recommendations.extend(recommended_item_ids)

    if not is_finished:
        print("Recommendations:")
        for item_id in recommendations:
            print(item_id)

# Example usage
user_id = 1
num_recommendations = 10
num_rounds = 3

multi_round_recommendation(user_id, num_recommendations, num_rounds)

AttributeError: ignored

In [ ]:

# Find the maximum user and movie IDs
max_user_id = ratings['user_id'].max()
max_movie_id = ratings['item_id'].max()


# Create a user-item matrix
user_item_matrix = np.zeros((max_user_id, max_movie_id))
for user_id, item_id, rating in ratings.values:
    user_item_matrix[user_id - 1, item_id - 1] = rating

# Define item importance weights
item_importance_weights = tf.Variable(tf.ones(len(ratings['item_id'].unique())))

# Define attention modeling with feedback-based neural network
def attention_model(user_item_matrix, item_ids, item_importance_weights):
    # Embed item IDs
    item_embedding_layer = tf.keras.layers.Embedding(input_dim=len(ratings['item_id'].unique()), output_dim=16)
    item_embeddings = item_embedding_layer(item_ids)

    # Transpose item embeddings twice to match user item matrix shape
    transposed_item_embeddings = tf.transpose(item_embeddings)
    transposed_item_embeddings = tf.transpose(transposed_item_embeddings)

    # Calculate attention scores using neural network
    user_item_embeddings = tf.matmul(user_item_matrix[np.newaxis, :], transposed_item_embeddings)

    # Calculate attention scores using neural network
    # user_item_embeddings = tf.matmul(user_item_matrix[np.newaxis, :], item_embeddings)
    attention_scores = tf.nn.softmax(tf.squeeze(user_item_embeddings, axis=0))

    # Apply item importance weights
    weighted_attention_scores = attention_scores * item_importance_weights

    return weighted_attention_scores

# Define multi-round recommendation function
def multi_round_recommendation(user_id, num_recommendations, num_rounds):
    recommendations = []

    is_finished = False
    for _ in range(num_rounds):
        # Recommend items based on attention model
        item_ids = np.array(range(len(ratings['item_id'].unique())))

        # Update item importance weights using neural network
        with tf.GradientTape() as tape:
            attention_scores = attention_model(user_item_matrix[user_id - 1, :], item_ids, item_importance_weights)
            item_importance_gradients = tape.gradient(attention_scores, item_importance_weights)

        # Update item importance weights based on feedback
        item_importance_weights.assign_sub(item_importance_gradients * user_item_matrix[user_id - 1, :])

        # Recommend top n items
        recommended_item_ids = item_ids[np.argsort(attention_scores)[-num_recommendations:]]

        # Print recommended items
        print("Recommended items:")
        for item_id in recommended_item_ids:
            print(item_id)

        # Get user feedback
        selected_item_id = input("Select an item (or type 'end' to finish): ")

        if selected_item_id == 'end':
            is_finished = True
            break

        # Update user profile with new feedback
        user_item_matrix[user_id - 1, selected_item_id - 1] = 1

        # Add recommended items to the list
        recommendations.extend(recommended_item_ids)

    if not is_finished:
        print("Recommendations:")
        for item_id in recommendations:
            print(item_id)

# Example usage
user_id = 1
num_recommendations = 10
num_rounds = 3

multi_round_recommendation(user_id, num_recommendations, num_rounds)

TypeError: ignored

In [ ]:


# Create a user-item matrix
user_item_matrix = np.zeros((len(ratings['user_id'].unique()), len(ratings['item_id'].unique())))
for user_id, item_id, rating in ratings.values:
    user_item_matrix[user_id - 1, item_id - 1] = rating

# Define item importance weights
item_importance_weights = tf.Variable(tf.ones(len(ratings['item_id'].unique())))

# Define attention modeling with feedback-based neural network
def attention_model(user_item_matrix, item_ids, item_importance_weights):
    # Create item embedding matrix
    item_embedding_matrix = tf.Variable(tf.random.normal([len(ratings['item_id'].unique()), 16]))

    # Calculate item embeddings for selected items
    item_embeddings_selected = tf.cast(item_embedding_matrix[item_ids], tf.float64)

    # Calculate user-item similarity matrix
    user_item_embeddings = tf.matmul(user_item_matrix[np.newaxis, :], item_embeddings_selected)

    # Calculate attention scores using neural network
    attention_scores = tf.nn.softmax(tf.squeeze(user_item_embeddings, axis=0))

    # Apply item importance weights
    weighted_attention_scores = attention_scores * item_importance_weights

    return weighted_attention_scores

# Define multi-round recommendation function
def multi_round_recommendation(user_id, num_recommendations, num_rounds):
    recommendations = []

    is_finished = False
    for _ in range(num_rounds):
        # Recommend items based on attention model
        item_ids = np.array(range(len(ratings['item_id'].unique())))

        # Update item importance weights using neural network
        with tf.GradientTape() as tape:
            attention_scores = attention_model(user_item_matrix[user_id - 1, :], item_ids, item_importance_weights)
            item_importance_gradients = tape.gradient(attention_scores, item_importance_weights)

        # Update item importance weights based on feedback
        item_importance_weights.assign_sub(item_importance_gradients * user_item_matrix[user_id - 1, :])

        # Recommend top n items
        recommended_item_ids = item_ids[np.argsort(attention_scores)[-num_recommendations:]]

        # Print recommended items
        print("Recommended items:")
        for item_id in recommended_item_ids:
            print(item_id)

        # Get user feedback
        selected_item_id = input("Select an item (or type 'end' to finish): ")

        if selected_item_id == 'end':
            is_finished = True
            break

        # Update user profile with new feedback
        user_item_matrix[user_id - 1, selected_item_id - 1] = 1

        # Add recommended items to the list
        recommendations.extend(recommended_item_ids)

    if not is_finished:
        print("Recommendations:")
        for item_id in recommendations:
            print(item_id)

# Example usage
user_id = 1
num_recommendations = 10
num_rounds = 3

multi_round_recommendation(user_id, num_recommendations, num_rounds)

IndexError: ignored

In [ ]:

# Validate item IDs and filter out invalid ratings
valid_ratings = ratings[ratings['item_id'] <= ratings['item_id'].max()]
valid_ratings = valid_ratings[valid_ratings['item_id'] >= ratings['item_id'].min()]

# Create a user-item matrix
user_item_matrix = np.zeros((len(valid_ratings['user_id'].unique()), len(valid_ratings['item_id'].unique())))
for user_id, item_id, rating in valid_ratings.values:
    user_item_matrix[user_id - 1, item_id - 1] = rating

# Define item importance weights
item_importance_weights = tf.Variable(tf.ones(len(valid_ratings['item_id'].unique())))

# Define attention modeling with feedback-based neural network
def attention_model(user_item_matrix, item_ids, item_importance_weights):
    # Create item embedding matrix
    item_embedding_matrix = tf.Variable(tf.random.normal([len(valid_ratings['item_id'].unique()), 16]))

    # Calculate item embeddings for selected items
    item_embeddings_selected = tf.cast(item_embedding_matrix[item_ids], tf.float64)

    # Calculate user-item similarity matrix
    user_item_embeddings = tf.matmul(user_item_matrix[np.newaxis, :], item_embeddings_selected)

    # Calculate attention scores using neural network
    attention_scores = tf.nn.softmax(tf.squeeze(user_item_embeddings, axis=0))

    # Apply item importance weights
    weighted_attention_scores = attention_scores * item_importance_weights

    return weighted_attention_scores

# Define multi-round recommendation function
def multi_round_recommendation(user_id, num_recommendations, num_rounds):
    recommendations = []

    is_finished = False
    for _ in range(num_rounds):
        # Recommend items based on attention model
        item_ids = np.array(range(len(valid_ratings['item_id'].unique())))

        # Update item importance weights using neural network
        with tf.GradientTape() as tape:
            attention_scores = attention_model(user_item_matrix[user_id - 1, :], item_ids, item_importance_weights)
            item_importance_gradients = tape.gradient(attention_scores, item_importance_weights)

        # Update item importance weights based on feedback
        item_importance_weights.assign_sub(item_importance_gradients * user_item_matrix[user_id - 1, :])

        # Recommend top n items
        recommended_item_ids = item_ids[np.argsort(attention_scores)[-num_recommendations:]]

        # Print recommended items
        print("Recommended items:")
        for item_id in recommended_item_ids:
            print(item_id)

        # Get user feedback
        selected_item_id = input("Select an item (or type 'end' to finish): ")

        if selected_item_id == 'end':
            is_finished = True
            break

        # Update user profile with new feedback
        user_item_matrix[user_id - 1, selected_item_id - 1] = 1

        # Add recommended items to the list
        recommendations.extend(recommended_item_ids)

    if not is_finished:
        print("Recommendations:")
        for item_id in recommendations:
            print(item_id)

# Example usage
user_id = 1
num_recommendations = 10
num_rounds = 3

multi_round_recommendation(user_id, num_recommendations, num_rounds)

IndexError: ignored

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

# Load Movielens 100k dataset
data_path = 'ml-100k/'
ratings = pd.read_csv(data_path + 'u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
ratings = ratings.drop('timestamp', axis=1)

# Create a user-item matrix
user_item_matrix = np.zeros((len(ratings['user_id'].unique()), len(ratings['item_id'].unique())))
for user_id, item_id, rating in ratings.values:
    user_item_matrix[user_id - 1, item_id - 1] = rating

# Define item importance weights
item_importance_weights = tf.Variable(tf.ones(len(ratings['item_id'].unique()), dtype=tf.float32))

# Define attention modeling with feedback-based neural network
def attention_model(user_item_matrix, item_ids, item_importance_weights):
    # Create item embedding matrix
    item_embedding_matrix = tf.Variable(tf.random.normal([len(ratings['item_id'].unique()), 16]))

    # Calculate item embeddings for selected items
    item_embeddings_selected = tf.cast(item_embedding_matrix[item_ids], tf.float32)

    # Calculate user-item similarity matrix
    user_item_embeddings = tf.matmul(user_item_matrix[np.newaxis, :], item_embeddings_selected)

    # Calculate attention scores using neural network
    attention_scores = tf.nn.softmax(tf.squeeze(user_item_embeddings, axis=0))

    # Apply item importance weights
    weighted_attention_scores = attention_scores * item_importance_weights

    return weighted_attention_scores

# Define multi-round recommendation function
def multi_round_recommendation(user_id, num_recommendations, num_rounds):
    recommendations = []

    is_finished = False
    for _ in range(num_rounds):
        # Recommend items based on attention model
        item_ids = np.array(range(len(ratings['item_id'].unique())))

        # Calculate attention scores and item importance gradients
        with tf.GradientTape() as tape:
            attention_scores = attention_model(user_item_matrix, item_ids, item_importance_weights)
            item_importance_gradients = tape.gradient(attention_scores, item_importance_weights)

        # Update item importance weights based on feedback
        if item_importance_gradients is not None:
            for item_id, item_rating, item_importance_gradient in zip(item_ids, item_ratings, item_importance_gradients):
                item_importance_weights.assign_sub(item_importance_gradient * item_rating)


In [ ]:


# Define multi-round recommendation function
def multi_round_recommendation(user_id, num_recommendations, num_rounds):
    print("inside multround")
    recommendations = []

    is_finished = False
    for _ in range(num_rounds):
        # Get user profile for current round
        current_user_profile = user_profiles[user_id]
        print(user_profiles[user_id])
        print(current_user_profile)

        # Recommend items based on attention model
        item_ids = np.array(list(current_user_profile.keys()))
        item_ratings = np.array(list(current_user_profile.values()))

        print("itemID",item_ids)
        print("itemRatings",item_ratings)


        # # Update item importance weights using neural network
        # with tf.GradientTape() as tape:
        #     print("in a loop of item importance updation")
        #     attention_scores = attention_model(current_user_profile, item_ids, item_importance_weights)
        #     print("Attention Scores from a func")
        #     print(attention_scores)
        #     print(attention_scores.dtype)
        #     print(item_importance_weights)
        #     item_importance_gradients = tape.gradient(attention_scores, item_importance_weights)
        #     print("Grad")
        #     print(item_importance_gradients)

                # Calculate the attention scores using the item importance weights
        # item_importance_weights = np.zeros(len(item_profiles[user_id]))
        # attention_scores = attention_model(current_user_profile, item_ids, item_importance_weights)
        learning_rate= 0.01
        epsilon= 0.001
        # Calculate the approximate gradient
        item_importance_gradients = []
        for i in range(len(ratings['item_id'])):
          item_importance_weights_perturbed = item_importance_weights.copy()
          item_importance_weights_perturbed[i] += epsilon
          attention_scores_perturbed = attention_model(current_user_profile, item_ids, item_importance_weights_perturbed)
          item_importance_gradients.append((attention_scores_perturbed[i] - attention_scores_perturbed [i]) / epsilon)

        # Update the item importance weights
        item_importance_weights -= learning_rate * item_importance_gradients
        # Update item importance weights based on feedback
        for item_id, item_rating, item_importance_gradient in zip(item_ids, item_ratings, item_importance_gradients):
            item_importance_weights.assign_sub(item_importance_gradient * item_rating)

        # Recommend top n items
        recommended_item_ids = item_ids[np.argsort(attention_scores_perturbed)[-num_recommendations:]]

        print("Recommend: ",recommended_item_ids)

        # Print recommended items
        print("Recommended items:")
        for item_id in recommended_item_ids:
            print(item_id)

        # Get user feedback
        selected_item_id = input("Select an item (or type 'end' to finish): ")

        if selected_item_id == 'end':
            is_finished = True
            break

        # Update user profile with new feedback
        current_user_profile[selected_item_id] = 1

        # Add recommended items to the list
        recommendations.extend(recommended_item_ids)

    if not is_finished:
        print("Recommendations:")
        for item_id in recommendations:
            print(item_id)


In [ ]:
from zipfile import ZipFile
from urllib.request import urlretrieve
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score, recall_score
from sklearn.decomposition import NMF
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import ndcg_score
from tensorflow.keras.layers import Input, Embedding, Dense, GlobalAveragePooling1D,  MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm

In [ ]:
## extracting data
urlretrieve("http://files.grouplens.org/datasets/movielens/ml-1m.zip", "movielens.zip")
ZipFile("movielens.zip", "r").extractall()

In [ ]:
data = pd.read_csv(
    "ml-1m/ratings.dat",
    sep="::",
    names=["userId", "movieId", "rating", "unix_timestamp"],
)
data.head()

<ipython-input-3-7c58046d1af1>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(


,userId,movieId,rating,unix_timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:

users = data['userId']
movies = data['movieId']
ratings = data['rating']

# Encode user and movie IDs
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
users = user_encoder.fit_transform(users)
movies = movie_encoder.fit_transform(movies)

# Split data into training and testing sets
users_train, users_test, movies_train, movies_test, ratings_train, ratings_test = train_test_split(
    users, movies, ratings, test_size=0.2, random_state=42
)

# Model parameters
embedding_dim = 32
num_heads = 4
num_blocks = 2

# Define the model
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

user_embedding = Embedding(input_dim=len(user_encoder.classes_), output_dim=embedding_dim)(user_input)
movie_embedding = Embedding(input_dim=len(movie_encoder.classes_), output_dim=embedding_dim)(movie_input)

# user_movie_attention = Attention(use_scale=True, num_heads=num_heads)([user_embedding, movie_embedding])

# user_movie_representation = GlobalAveragePooling1D()(user_movie_attention)

user_movie_attention = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)([user_embedding, movie_embedding])
user_movie_representation = GlobalAveragePooling1D()(user_movie_attention)

output = Dense(1, activation='linear')(user_movie_representation)

model = Model(inputs=[user_input, movie_input], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Set parameters for multi-round recommendation
num_rounds = 5  # Number of recommendation rounds
num_recommendations = 10  # Number of recommendations to provide in each round

# Simulate multi-round recommendation
for round_num in range(1, num_rounds + 1):
    print(f"\nRound {round_num}:")

    # Train the model
    model.fit([users_train, movies_train], ratings_train, epochs=1, batch_size=64, validation_split=0.1)

    # Generate top-N recommendations for each user
    all_user_ids = np.unique(users_test)
    all_movie_ids = np.unique(movies_test)
    all_predictions = model.predict([all_user_ids, all_movie_ids])

    # Provide top-N recommendations to each user
    precision_scores = []
    recall_scores = []
    ndcg_scores = []

    for user_id in all_user_ids:
        top_n_recommendations = np.argsort(all_predictions[users_test == user_id])[::-1][:num_recommendations]

        # Simulate user feedback (for demonstration purposes, you can replace this with actual user feedback)
        user_feedback = np.random.choice(top_n_recommendations, size=int(num_recommendations / 2), replace=False)
        user_ratings = np.random.choice([4, 5], size=int(num_recommendations / 2))  # Simulated positive feedback

        # Update the model with user feedback
        user_indices = np.where(users_test == user_id)[0]
        movie_indices = np.where(np.isin(movies_test[user_indices], user_feedback))[0]
        model.fit([users_test[user_indices[movie_indices]], movies_test[user_indices[movie_indices]]],
                  ratings_test[user_indices[movie_indices]], epochs=1, batch_size=64)

        # Evaluate the model after feedback
        true_movies = movies_test[users_test == user_id]
        true_ratings = ratings_test[users_test == user_id]

        # Get top-N recommendations for the user
        top_n_recommendations = np.argsort(all_predictions[users_test == user_id])[::-1][:num_recommendations]

        # Precision@k
        precision = precision_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')
        precision_scores.append(precision)

        # Recall@k
        recall = recall_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')
        recall_scores.append(recall)

        # NDCG
        ndcg = ndcg_score([true_movies], [top_n_recommendations], k=num_recommendations)
        ndcg_scores.append(ndcg)

    # Calculate and print average scores after each round
    avg_precision = np.mean(precision_scores)
    avg_recall = np.mean(recall_scores)
    avg_ndcg = np.mean(ndcg_scores)

    print(f'Precision@{num_recommendations}: {avg_precision}')
    print(f'Recall@{num_recommendations}: {avg_recall}')
    print(f'NDCG@{num_recommendations}: {avg_ndcg}')

# Evaluate the final model
precision_scores = []
recall_scores = []
ndcg_scores = []

for user_id in tqdm(all_user_ids):
    true_movies = movies_test[users_test == user_id]
    true_ratings = ratings_test[users_test == user_id]

    # Get top-N recommendations for the user
    top_n_recommendations = np.argsort(all_predictions[users_test == user_id])[::-1][:num_recommendations]

    # Precision@k
    precision = precision_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')
    precision_scores.append(precision)

    # Recall@k
    recall = recall_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')
    recall_scores.append(recall)

    # NDCG
    ndcg = ndcg_score([true_movies], [top_n_recommendations], k=num_recommendations)
    ndcg_scores.append(ndcg)

# Calculate and print average scores for the final model
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)
avg_ndcg = np.mean(ndcg_scores)

print(f'Final Model Evaluation:')
print(f'Precision@{num_recommendations}: {avg_precision}')
print(f'Recall@{num_recommendations}: {avg_recall}')
print(f'NDCG@{num_recommendations}: {avg_ndcg}')


TypeError: ignored

In [ ]:
# Encode user and movie IDs
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
users = user_encoder.fit_transform(users)
movies = movie_encoder.fit_transform(movies)

# Split data into training and testing sets
users_train, users_test, movies_train, movies_test, ratings_train, ratings_test = train_test_split(
    users, movies, ratings, test_size=0.2, random_state=42
)

# Model parameters
embedding_dim = 32
num_heads = 4
num_blocks = 2

# Define the model
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

user_embedding = Embedding(input_dim=len(user_encoder.classes_), output_dim=embedding_dim)(user_input)
movie_embedding = Embedding(input_dim=len(movie_encoder.classes_), output_dim=embedding_dim)(movie_input)

# Create value tensor with the same shape as query tensor
value_embedding = Embedding(input_dim=len(movie_encoder.classes_), output_dim=embedding_dim)(movie_input)

# Create value tensor with the same shape as query tensor

user_movie_attention = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(user_embedding, movie_embedding, value_embedding)
user_movie_representation = GlobalAveragePooling1D()(user_movie_attention)


output = Dense(1, activation='linear')(user_movie_representation)

model = Model(inputs=[user_input, movie_input], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Set parameters for multi-round recommendation
num_rounds = 5  # Number of recommendation rounds
num_recommendations = 10  # Number of recommendations to provide in each round

In [ ]:
# Simulate multi-round recommendation
for round_num in range(1, num_rounds + 1):
    print(f"\nRound {round_num}:")

    # Train the model
    model.fit([users_train, movies_train], ratings_train, epochs=1, batch_size=64, validation_split=0.1)

    # Generate top-N recommendations for each user
    all_user_ids = np.unique(users_test)
    all_movie_ids = np.unique(movies_test)
    all_predictions = model.predict([all_user_ids, all_movie_ids])

    # Provide top-N recommendations to each user
    precision_scores = []
    recall_scores = []
    ndcg_scores = []

    for user_id in all_user_ids:
        top_n_recommendations = np.argsort(all_predictions[users_test == user_id])[::-1][:num_recommendations]

        # Simulate user feedback (for demonstration purposes, you can replace this with actual user feedback)
        user_feedback = np.random.choice(top_n_recommendations, size=int(num_recommendations / 2), replace=False)
        user_ratings = np.random.choice([4, 5], size=int(num_recommendations / 2))  # Simulated positive feedback

        # Update the model with user feedback
        user_indices = np.where(users_test == user_id)[0]
        movie_indices = np.where(np.isin(movies_test[user_indices], user_feedback))[0]
        model.fit([users_test[user_indices[movie_indices]], movies_test[user_indices[movie_indices]]],
                  ratings_test[user_indices[movie_indices]], epochs=1, batch_size=64)

        # Evaluate the model after feedback
        true_movies = movies_test[users_test == user_id]
        true_ratings = ratings_test[users_test == user_id]

        # Get top-N recommendations for the user
        top_n_recommendations = np.argsort(all_predictions[users_test == user_id])[::-1][:num_recommendations]

        # Precision@k
        precision = precision_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')
        precision_scores.append(precision)

        # Recall@k
        recall = recall_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')
        recall_scores.append(recall)

        # NDCG
        ndcg = ndcg_score([true_movies], [top_n_recommendations], k=num_recommendations)
        ndcg_scores.append(ndcg)

    # Calculate and print average scores after each round
    avg_precision = np.mean(precision_scores)
    avg_recall = np.mean(recall_scores)
    avg_ndcg = np.mean(ndcg_scores)

    print(f'Precision@{num_recommendations}: {avg_precision}')
    print(f'Recall@{num_recommendations}: {avg_recall}')
    print(f'NDCG@{num_recommendations}: {avg_ndcg}')

# Evaluate the final model
precision_scores = []
recall_scores = []
ndcg_scores = []

for user_id in tqdm(all_user_ids):
    true_movies = movies_test[users_test == user_id]
    true_ratings = ratings_test[users_test == user_id]

    # Get top-N recommendations for the user
    top_n_recommendations = np.argsort(all_predictions[users_test == user_id])[::-1][:num_recommendations]

    # Precision@k
    precision = precision_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')
    precision_scores.append(precision)

    # Recall@k
    recall = recall_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')
    recall_scores.append(recall)

    # NDCG
    ndcg = ndcg_score([true_movies], [top_n_recommendations], k=num_recommendations)
    ndcg_scores.append(ndcg)

# Calculate and print average scores for the final model
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)
avg_ndcg = np.mean(ndcg_scores)

print(f'Final Model Evaluation:')
print(f'Precision@{num_recommendations}: {avg_precision}')
print(f'Recall@{num_recommendations}: {avg_recall}')
print(f'NDCG@{num_recommendations}: {avg_ndcg}')



Round 1:
11253/11253 [==============================] - 62s 6ms/step - loss: 0.9724 - val_loss: 0.9718


ValueError: ignored

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import ndcg_score
from tensorflow.keras.layers import Input, Embedding, Dense, GlobalAveragePooling1D, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm

# Load MovieLens dataset
# Assuming 'ratings.csv' contains columns: userId, movieId, rating
data = data
users = data['userId']
movies = data['movieId']
ratings = data['rating']

# Encode user and movie IDs
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
users = user_encoder.fit_transform(users)
movies = movie_encoder.fit_transform(movies)

# Split data into training and testing sets
users_train, users_test, movies_train, movies_test, ratings_train, ratings_test = train_test_split(
    users, movies, ratings, test_size=0.2, random_state=42
)

# Model parameters
embedding_dim = 32
num_heads = 4
num_blocks = 2

# Define the model
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

user_embedding = Embedding(input_dim=len(user_encoder.classes_), output_dim=embedding_dim)(user_input)
movie_embedding = Embedding(input_dim=len(movie_encoder.classes_), output_dim=embedding_dim)(movie_input)

# Separate Embedding layer for user profiles
user_profile_embedding = Embedding(input_dim=len(user_encoder.classes_), output_dim=embedding_dim)(user_input)

# Use user profile embedding as value tensor
user_movie_attention = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(user_embedding, movie_embedding, user_profile_embedding)
user_movie_representation = GlobalAveragePooling1D()(user_movie_attention)

output = Dense(1, activation='linear')(user_movie_representation)

model = Model(inputs=[user_input, movie_input], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Set parameters for multi-round recommendation
num_rounds = 5  # Number of recommendation rounds
num_recommendations = 10  # Number of recommendations to provide in each round

# Simulate multi-round recommendation
for round_num in range(1, num_rounds + 1):
    print(f"\nRound {round_num}:")

    # Train the model
    model.fit([users_train, movies_train], ratings_train, epochs=1, batch_size=64, validation_split=0.1)

    # Generate top-N recommendations for each user
    all_user_ids = np.unique(users_test)
    all_movie_ids = np.unique(movies_test)
    # all_predictions = model.predict([all_user_ids, all_movie_ids])

    # Create a meshgrid of all user IDs and movie IDs
    all_user_ids_mesh, all_movie_ids_mesh = np.meshgrid(all_user_ids, all_movie_ids)

    all_predictions = model.predict([all_user_ids_mesh.flatten(), all_movie_ids_mesh.flatten()])

    # Provide top-N recommendations to each user
    precision_scores = []
    recall_scores = []
    ndcg_scores = []

    for user_id in all_user_ids:
        top_n_recommendations = np.argsort(all_predictions[users_test == user_id])[::-1][:num_recommendations]

        # Simulate user feedback (for demonstration purposes, you can replace this with actual user feedback)
        user_feedback = np.random.choice(top_n_recommendations, size=int(num_recommendations / 2), replace=False)
        user_ratings = np.random.choice([4, 5], size=int(num_recommendations / 2))  # Simulated positive feedback

        # Update the model with user feedback
        user_indices = np.where(users_test == user_id)[0]
        movie_indices = np.where(np.isin(movies_test[user_indices], user_feedback))[0]
        model.fit([users_test[user_indices[movie_indices]], movies_test[user_indices[movie_indices]]],
                  ratings_test[user_indices[movie_indices]], epochs=1, batch_size=64)

        # Evaluate the model after feedback
        true_movies = movies_test[users_test == user_id]
        true_ratings = ratings_test[users_test == user_id]

        # Get top-N recommendations for the user
        top_n_recommendations = np.argsort(all_predictions[users_test == user_id])[::-1][:num_recommendations]

        # Precision@k
        precision = precision_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')
        precision_scores.append(precision)

        # Recall@k
        recall = recall_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')
        recall_scores.append(recall)

        # NDCG
        ndcg = ndcg_score([true_movies], [top_n_recommendations], k=num_recommendations)
        ndcg_scores.append(ndcg)

    # Calculate and print average scores after each round
    avg_precision = np.mean(precision_scores)
    avg_recall = np.mean(recall_scores)
    avg_ndcg = np.mean(ndcg_scores)

    print(f'Precision@{num_recommendations}: {avg_precision}')
    print(f'Recall@{num_recommendations}: {avg_recall}')
    print(f'NDCG@{num_recommendations}: {avg_ndcg}')

# Evaluate the final model
precision_scores = []
recall_scores = []
ndcg_scores = []

for user_id in tqdm(all_user_ids):
    true_movies = movies_test[users_test == user_id]
    true_ratings = ratings_test[users_test == user_id]

    # Get top-N recommendations for the user
    top_n_recommendations = np.argsort(all_predictions[users_test == user_id])[::-1][:num_recommendations]

    # Precision@k
    precision = precision_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')
    precision_scores.append(precision)

    # Recall@k
    recall = recall_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')
    recall_scores.append(recall)

    # NDCG
    ndcg = ndcg_score([true_movies], [top_n_recommendations], k=num_recommendations)
    ndcg_scores.append(ndcg)

# Calculate and print average scores for the final model
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)
avg_ndcg = np.mean(ndcg_scores)

print(f'Final Model Evaluation:')
print(f'Precision@{num_recommendations}: {avg_precision}')
print(f'Recall@{num_recommendations}: {avg_recall}')
print(f'NDCG@{num_recommendations}: {avg_ndcg}')



Round 1:
649840/649840 [==============================] - 862s 1ms/step


IndexError: ignored

In [ ]:
# Load MovieLens dataset
# Assuming 'ratings.csv' contains columns: userId, movieId, rating
data = data
users = data['userId']
movies = data['movieId']
ratings = data['rating']

# Encode user and movie IDs
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
users = user_encoder.fit_transform(users)
movies = movie_encoder.fit_transform(movies)

# Model parameters
embedding_dim = 32
num_heads = 4
num_blocks = 2

# Define the model
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

user_embedding = Embedding(input_dim=len(user_encoder.classes_), output_dim=embedding_dim)(user_input)
movie_embedding = Embedding(input_dim=len(movie_encoder.classes_), output_dim=embedding_dim)(movie_input)

# Separate Embedding layer for user profiles
user_profile_embedding = Embedding(input_dim=len(user_encoder.classes_), output_dim=embedding_dim)(user_input)

# Use user profile embedding as value tensor
user_movie_attention = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(user_embedding, movie_embedding, user_profile_embedding)
user_movie_representation = GlobalAveragePooling1D()(user_movie_attention)

output = Dense(1, activation='linear')(user_movie_representation)

model = Model(inputs=[user_input, movie_input], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Set parameters for multi-round recommendation
num_rounds = 5  # Number of recommendation rounds
num_recommendations = 10  # Number of recommendations to provide in each round

# Select a specific user for evaluation
specific_user_id = np.random.choice(users, size=1)[0]

# Simulate multi-round recommendation and feedback for the specific user
for round_num in range(1, num_rounds + 1):
    print(f"\nRound {round_num}:")

    # Train the model
    model.fit([users, movies], ratings, epochs=1, batch_size=64)

    # Generate top-N recommendations for the specific user
    user_indices = np.where(users == specific_user_id)[0]
    all_movie_ids = np.unique(movies)
    all_movie_ids_mesh = np.tile(all_movie_ids, len(user_indices))

    all_predictions = model.predict([np.repeat(specific_user_id, len(all_movie_ids_mesh)), all_movie_ids_mesh])

    top_n_recommendations = np.argsort(all_predictions[:, 0])[::-1][:num_recommendations]

    # Simulate user feedback (for demonstration purposes, you can replace this with actual user feedback)
    user_feedback = np.random.choice(top_n_recommendations, size=int(num_recommendations / 2), replace=False)
    user_ratings = np.random.choice([4, 5], size=int(num_recommendations / 2))  # Simulated positive feedback

    # Update the model with user feedback
    model.fit([np.repeat(specific_user_id, len(user_feedback)), user_feedback], user_ratings, epochs=1, batch_size=64)


    # Evaluate the model after feedback
    true_movies = movies[users == specific_user_id]
    true_ratings = ratings[users == specific_user_id]

    # Get top-N recommendations for the user
    top_n_recommendations = np.argsort(all_predictions[:, 0])[::-1][:num_recommendations]

    # Precision@k
    precision = precision_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')

    # Recall@k
    recall = recall_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')

    # NDCG
    ndcg = ndcg_score([true_movies], [top_n_recommendations], k=num_recommendations)

    # Print evaluation metrics
    print(f'Precision@{num_recommendations}: {precision}')
    print(f'Recall@{num_recommendations}: {recall}')
    print(f'NDCG@{num_recommendations}: {ndcg}')


Round 1:
57096/57096 [==============================] - 84s 1ms/step


InvalidArgumentError: ignored

In [ ]:
# Load MovieLens dataset
# Assuming 'ratings.csv' contains columns: userId, movieId, rating
data = data
users = data['userId']
movies = data['movieId']
ratings = data['rating']

# Encode user and movie IDs
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
users = user_encoder.fit_transform(users)
movies = movie_encoder.fit_transform(movies)

# Model parameters
embedding_dim = 32
num_heads = 4

# Define the model
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

user_embedding = Embedding(input_dim=len(user_encoder.classes_), output_dim=embedding_dim)(user_input)
movie_embedding = Embedding(input_dim=len(movie_encoder.classes_), output_dim=embedding_dim)(movie_input)

# Separate Embedding layer for user profiles
user_profile_embedding = Embedding(input_dim=len(user_encoder.classes_), output_dim=embedding_dim)(user_input)

# Use user profile embedding as value tensor
user_movie_attention = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(user_embedding, movie_embedding, user_profile_embedding)
user_movie_representation = GlobalAveragePooling1D()(user_movie_attention)

output = Dense(1, activation='linear')(user_movie_representation)

model = Model(inputs=[user_input, movie_input], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Set parameters for multi-round recommendation
num_rounds = 5  # Number of recommendation rounds
num_recommendations = 10  # Number of recommendations to provide in each round

# Select a specific user for evaluation
specific_user_id = np.random.choice(users, size=1)[0]

# Simulate multi-round recommendation and feedback for the specific user
for round_num in range(1, num_rounds + 1):
    print(f"\nRound {round_num}:")

    # Train the model
    model.fit([users, movies], ratings, epochs=1, batch_size=64, shuffle=True)

    # Generate top-N recommendations for the specific user
    all_movie_ids = np.unique(movies)
    all_movie_ids_mesh = np.tile(all_movie_ids, len(users))

    all_predictions = model.predict([users, all_movie_ids_mesh])

    top_n_recommendations = np.argsort(all_predictions[:, 0])[::-1][:num_recommendations]

    # Simulate user feedback (for demonstration purposes, you can replace this with actual user feedback)
    user_feedback = np.random.choice(top_n_recommendations, size=int(num_recommendations / 2), replace=False)
    user_ratings = np.random.choice([4, 5], size=int(num_recommendations / 2))  # Simulated positive feedback

    # Update the model with user feedback
    model.fit([np.repeat(specific_user_id, len(user_feedback)), user_feedback], user_ratings, epochs=1, batch_size=64)

    # Evaluate the model after feedback
    true_movies = movies[users == specific_user_id]
    true_ratings = ratings[users == specific_user_id]

    # Get top-N recommendations for the user
    top_n_recommendations = np.argsort(all_predictions[:, 0])[::-1][:num_recommendations]

    # Precision@k
    precision = precision_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')

    # Recall@k
    recall = recall_score(true_movies, np.isin(top_n_recommendations, true_movies), average='binary')

    # NDCG
    ndcg = ndcg_score([true_movies], [top_n_recommendations], k=num_recommendations)

    # Print evaluation metrics
    print(f'Precision@{num_recommendations}: {precision}')
    print(f'Recall@{num_recommendations}: {recall}')
    print(f'NDCG@{num_recommendations}: {ndcg}')



Round 1:
15629/15629 [==============================] - 104s 7ms/step - loss: 1.0122
